In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import math
import numpy as np
import pickle
import random
import time
from tqdm import tqdm
from datetime import datetime

from stgnn import Model
from stgnn_transformer import Model as Model2
# from stgnn_handler import ForecastDataset, train, test
# from utils import convert_df_wavelet_input, dl_preprocess_data_test
# from utils import rmse, mape, nrmse, nrmse2

import torch
import torch.utils.data as torch_data
import torch.nn as nn
from tensorflow.random import set_seed
from numpy.random import seed
import shap

%matplotlib inline

In [3]:
base_dir = "/recsys_data/RecSys/fashion/polyvore-dataset/polyvore_outfits"
train_dir = os.path.join(base_dir, "disjoint")
image_dir = os.path.join(base_dir, "images")
train_json = "train.json"
valid_json = "valid.json"
test_json = "test.json"

train_file = "compatibility_train.txt"
valid_file = "compatibility_valid.txt"
test_file = "compatibility_test.txt"
item_file = "polyvore_item_metadata.json"
outfit_file = "polyvore_outfit_titles.json"

model_type = "rnn" #"set-transformer"
include_text = True
batch_size = 32

## Read all the required files

In [4]:
with open(os.path.join(train_dir, train_json), 'r') as fr:
    train_pos = json.load(fr)
    
with open(os.path.join(train_dir, valid_json), 'r') as fr:
    valid_pos = json.load(fr)
    
with open(os.path.join(train_dir, test_json), 'r') as fr:
    test_pos = json.load(fr)
    
with open(os.path.join(base_dir, item_file), 'r') as fr:
    pv_items = json.load(fr)
    
with open(os.path.join(base_dir, outfit_file), 'r') as fr:
    pv_outfits = json.load(fr)


In [5]:
with open(os.path.join(train_dir, train_file), 'r') as fr:
    train_X, train_y = [], []
    for line in fr:
        elems = line.strip().split()
        train_y.append(elems[0])
        train_X.append(elems[1:])

with open(os.path.join(train_dir, valid_file), 'r') as fr:
    valid_X, valid_y = [], []
    for line in fr:
        elems = line.strip().split()
        valid_y.append(elems[0])
        valid_X.append(elems[1:])

with open(os.path.join(train_dir, test_file), 'r') as fr:
    test_X, test_y = [], []
    for line in fr:
        elems = line.strip().split()
        test_y.append(elems[0])
        test_X.append(elems[1:])


### Create a dict that maps to real item-id

In [6]:
item_dict = {}
for ii, outfit in enumerate(train_pos):
    items = outfit['items']
    mapped = train_X[ii]
    item_dict.update({jj:kk['item_id'] for jj, kk in zip(mapped, items)})
print(len(item_dict))

for ii, outfit in enumerate(valid_pos):
    items = outfit['items']
    mapped = valid_X[ii]
    item_dict.update({jj:kk['item_id'] for jj, kk in zip(mapped, items)})
print(len(item_dict))

for ii, outfit in enumerate(test_pos):
    items = outfit['items']
    mapped = test_X[ii]
    item_dict.update({jj:kk['item_id'] for jj, kk in zip(mapped, items)})
print(len(item_dict))

85935
101223
175485


In [23]:
from stgnn_transformer import Model as Model2
from stgnn_transformer import BaseTransformer

In [8]:
params_set = {}
params_set['set1'] = {'order': 4, 
                      'factor': 3,
                      'dropout': 0.05,
                      'd_model': 32,
                      'num_heads': 4, 
                      'e_layers': 1, 
                      'dff': 128,
                      'only_transformer': True,
                      'use_fft_transformer': True,
                      'observe': 'rmse',
                      'patience': 15,
                      'lr': 1e-05}

params_set['set2'] = {'order': 4, 
                      'factor': 3,
                      'dropout': 0.43655976679400943,
                      'd_model': 96,
                      'num_heads': 4, 
                      'e_layers': 2, 
                      'dff': 180,
                      'only_transformer': True,
                      'use_fft_transformer': True,
                      'observe': 'rmse',
                      'patience': 15,
                      'lr': 1e-05}

params_set['set3'] = {'order': 4, 
                      'factor': 3,
                      'dropout': 0.05,
                      'd_model': 32,
                      'num_heads': 4, 
                      'e_layers': 1, 
                      'dff': 128,
                      'only_transformer': True,
                      'use_fft_transformer': False,
                      'observe': 'rmse',
                      'patience': 15,
                      'lr': 1e-05}

params_set['set4'] = {'order': 4, 
                      'factor': 3,
                      'dropout': 0.05,
                      'd_model': 32,
                      'num_heads': 4, 
                      'e_layers': 2, 
                      'dff': 128,
                      'only_transformer': True,
                      'use_fft_transformer': False,
                      'observe': 'rmse',
                      'patience': 15,
                      'lr': 1e-05}


In [25]:
setnum = 'set4'
# print(params_set[setnum])
model_params = {
                'lookback': 16,
                'horizon': 1,
                'batch_size': 32,
                'lr': 1e-04,
                'epochs': 100,
                'scheduler': None,
                'small_epochs': 1,
                'output_dir': './',
                'norm_method': 'min_max_m1',
                'exponential_decay_step':5,
                'validate_freq':1,
                'early_stop':True,
                'device': 'cuda',
            }
node_cnt = 1280
device = model_params['device']

# model = Model2(units=node_cnt,
#                stack_cnt=1,
#                time_step=model_params['lookback'],
#                multi_layer=2,
#                out_feature=1,
#                horizon=model_params['horizon'],
#                encoder2='fc',
#                order=params_set[setnum]['order'],  # tune parameter
#                factor=params_set[setnum]['factor'],
#                dropout=params_set[setnum]['dropout'],  # tune parameter, 0.05
#                d_model=params_set[setnum]['d_model'],
#                n_heads=params_set[setnum]['num_heads'],
#                e_layers=params_set[setnum]['e_layers'],
#                d_ff=params_set[setnum]['dff'],
#                only_transformer=params_set[setnum]['only_transformer'],
#                use_fft_transformer=params_set[setnum]['use_fft_transformer'],
#              )

model = BaseTransformer(1080, 768, 
                        num_layers=1,
                        d_model=64,
                        num_heads=1,
                        dff=32,
                        rate=0.0,
                        num_classes=2,
                        lstm_dim=32,
                        device=device,
                        embedding_activation="linear",
                        encoder_activation="relu",
                        lstm_activation="linear",
                        final_activation="sigmoid")

total_params = 0
for name, parameter in model.named_parameters():
    if not parameter.requires_grad:
        continue
    param = parameter.numel()
    total_params += param
print(f"Total Trainable Params: {total_params}")
model.to(device)

Total Trainable Params: 202113


BaseTransformer(
  (image_projector): Linear(in_features=1080, out_features=64, bias=True)
  (image_transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=32, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=32, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (text_projector): Linear(in_features=768, out_features=64, bias=True)
  (text_transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAt

In [21]:
from utils_torch import CustomDataset, train

In [11]:
train_set = CustomDataset(train_X, 
                          train_y, 
                          item_dict, 
                          pv_items, 
                          image_dir, 
                          batch_size=batch_size, 
                          only_image=not include_text)
valid_set = CustomDataset(valid_X, 
                          valid_y, 
                          item_dict, 
                          pv_items, 
                          image_dir, 
                          batch_size=batch_size, 
                          only_image=not include_text)

In [22]:
train(model, train_set, valid_set, device='cuda')

| Epoch   0 | time: 35.51s | train_loss 0.6934 | val-ACC 0.5013 | val-AUC 0.5001 ( 0)
| Epoch   1 | time: 35.02s | train_loss 0.6932 | val-ACC 0.5007 | val-AUC 0.5042 ( 1)
| Epoch   2 | time: 35.01s | train_loss 0.6933 | val-ACC 0.5012 | val-AUC 0.4966 ( 0)
| Epoch   3 | time: 35.05s | train_loss 0.6931 | val-ACC 0.4987 | val-AUC 0.4977 ( 1)
| Epoch   4 | time: 35.38s | train_loss 0.6934 | val-ACC 0.4992 | val-AUC 0.4988 ( 2)
| Epoch   5 | time: 35.53s | train_loss 0.6934 | val-ACC 0.5000 | val-AUC 0.4982 ( 3)
| Epoch   6 | time: 34.91s | train_loss 0.6933 | val-ACC 0.5000 | val-AUC 0.5035 ( 4)
| Epoch   7 | time: 35.53s | train_loss 0.6934 | val-ACC 0.4995 | val-AUC 0.4995 ( 5)


KeyboardInterrupt: 

In [27]:
import flash
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageEmbedder